 * @ Author: Yohei Ohto
 * @ Create Time: 2024-12-05 18:01:35
 * @ Modified time: 2024-12-05 18:01:46
 * @ Description: 日本の税制の理解と奨学金をもらう人の手取りを計算する　(東京大学、文京区、39歳以下、独身一人暮らし、バイト先が1つでの生活を想定)

In [3]:
from src import *

In [4]:
# 各種手当に対応していないため、バイトの場合、特に交通費はしっかりと抜いてから計算すること　https://edenred.jp/article/employee-benefits/119/#chapter-7

monthly = 0  # 奨学金の月額, 単位は万 (spring-GXの場合)
annual_kyuyo = monthly * 12 # 奨学金の年額, 単位は万

zatsu_shotoku = 18 # 雑所得の月額, 単位は万 (いくつかのwingsの場合)
keihi = 18 * 12 # 経費の月額, 単位は万 (交通費など)
annual_zatsu = zatsu_shotoku * 12 - keihi# 雑所得の年額,

part_job = 14 * 12 # バイトの年額, 単位は万
all_salary = annual_kyuyo + part_job + annual_zatsu # あなたの全額での収入金額 単位は万

print("私は一年で",all_salary,"万円稼ぎます!", flush=True)
print(f"給与 {part_job}万円 + 奨学金 {annual_kyuyo}万円 + 雑所得 {annual_zatsu}万円 = {all_salary}万円", flush=True)

私は一年で 168 万円稼ぎます!
給与 168万円 + 奨学金 0万円 + 雑所得 0万円 = 168万円


In [5]:
chihouzei_kiso_kojo = kiso_kojo_chiho(all_salary)
kokuzei_kiso_kojo = kiso_kojo_kuni(all_salary)
kyuyo_syotoku_kojo = kyuyo_syotoku_kojo_calc(all_salary)

print("地方税の基礎控除", chihouzei_kiso_kojo)
print("国税の基礎控除", kokuzei_kiso_kojo)
print("給与所得控除", kyuyo_syotoku_kojo)

# バイト先と奨学金という収入減が複数ある場合には主たる収入減において年末調整を行う義務がある → それぞれで給与所得控除が計算されているため、年末調整時に合算して追徴課税がある

地方税の基礎控除 43
国税の基礎控除 48
給与所得控除 57.2


In [6]:
# 国保 https://www.city.bunkyo.lg.jp/b021/p000424.html
# 年齢的に介護保険を支払う必要がないと想定
kiso_kojo = kiso_kojo_chiho(all_salary)
sante_kiso = all_salary - kyuyo_syotoku_kojo - kiso_kojo # 算定基礎額(所得) = 総所得(給与所得控除が計算済み) - 基礎控除

if sante_kiso < 0:
    sante_kiso = 0 # 基礎控除がマイナスになる場合は0として扱う

syotoku_wari_kiso = sante_kiso * (8.69 / 100) #〔所得割額〕被保険者全員の令和6年度の算定基礎額 × 8.69％
kinto_wari_kiso = 4.9100 * 1 # 〔均等割額〕49,100円 × 被保険者数 (独身想定の為1)
kiso_hoken = syotoku_wari_kiso + kinto_wari_kiso

print("基礎分保険料", kiso_hoken)

syotoku_wari_sien = sante_kiso * (2.80 / 100) #〔所得割額〕被保険者全員の令和6年度の算定基礎額 × 2.80％
kinto_wari_sien = 1.6500 * 1 # 〔均等割額〕16,500円 × 被保険者数(1)
sien_hoken = syotoku_wari_sien + kinto_wari_sien

print("支援金分保険料", sien_hoken)

kokuho = kiso_hoken + sien_hoken
monthly_kokuho = kokuho / 12

print("国保", kokuho, "万円, 月額", monthly_kokuho, "万円")


基礎分保険料 10.80182
支援金分保険料 3.5483999999999996
国保 14.350219999999998 万円, 月額 1.1958516666666665 万円


In [7]:
frequency = "当月末振替"
payment_method = "口座振替"
kokumin_nenkin = kokumin_nenkin_payment(frequency, payment_method)

monthly_kokumin_nenkin = kokumin_nenkin["月額"]
annual_kokumin_nenkin = kokumin_nenkin["年額"]

print("国民年金", annual_kokumin_nenkin, "万円, 月額", monthly_kokumin_nenkin, "万円")

国民年金 20.304 万円, 月額 1.692 万円


In [8]:
# 所得税 https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/2260.htm
# 定額減税 https://www.nta.go.jp/users/gensen/teigakugenzei/01.htm

kiso_kojo = kiso_kojo_kuni(all_salary)
kazei_taisyo =  all_salary - kyuyo_syotoku_kojo - kiso_kojo
income_tax = calculate_income_tax(kazei_taisyo)

if all_salary < 2000:
    income_tax -= 3

monthly_income_tax = income_tax / 12
print("所得税", income_tax, "万円, 月額", monthly_income_tax, "万円")

所得税 0.14000000000000012 万円, 月額 0.011666666666666678 万円


In [9]:
# 所得税に付随する税金　復興特別所得税、定額減税
hukko = income_tax * 0.021
monthly_hukko = hukko / 12
print("復興特別所得税", hukko, "万円, 月額", monthly_hukko, "万円")

復興特別所得税 0.002940000000000003 万円, 月額 0.00024500000000000026 万円


In [10]:
# 住民税　https://biz.moneyforward.com/tax_return/basic/49732/#1　https://www.city.bunkyo.lg.jp/b008/p000357.html#sonota　

kiso_kojo = kiso_kojo_chiho(all_salary)
kazei_taisyo_not_tyosei =  all_salary - kyuyo_syotoku_kojo - kiso_kojo - annual_kokumin_nenkin - kokuho

koujo = tyosei_koujo(kazei_taisyo_not_tyosei, kokuzei_kiso_kojo, chihouzei_kiso_kojo)
print("調整控除額", koujo)
kazei_taisyo = kazei_taisyo_not_tyosei - koujo
jumin_tax = kazei_taisyo * 0.1 + 0.45 + 0.05
monthly_jumin_tax = jumin_tax / 12
print("住民税", jumin_tax, "万円, 月額", monthly_jumin_tax, "万円")

調整控除額 0.25
住民税 3.7895779999999997 万円, 月額 0.31579816666666666 万円


In [11]:
annual_zeikin = jumin_tax + hukko + income_tax + annual_kokumin_nenkin + kokuho
print("あなたは1年で", annual_zeikin, "万円納税しています")

tedori = all_salary - annual_zeikin
print("手取りは", tedori, "万円です")

あなたは1年で 38.586738 万円納税しています
手取りは 129.413262 万円です


In [21]:
# 来年だけ
kokuho = monthly_kokuho * 5 # 7月末に超える → 8月文から国保に入る　https://mynavi-ms.jp/magazine/detail/001339.html
income_tax = income_tax
nenkin = monthly_kokuho * 5 # 8月分から年金を払い始める

kiso_kojo = kiso_kojo_chiho(all_salary)
kazei_taisyo_not_tyosei =  all_salary - kyuyo_syotoku_kojo - kiso_kojo - nenkin - kokuho
koujo = tyosei_koujo(kazei_taisyo_not_tyosei, kokuzei_kiso_kojo, chihouzei_kiso_kojo)
print("調整控除額", koujo)
kazei_taisyo = kazei_taisyo_not_tyosei - koujo
jumin_tax = kazei_taisyo * 0.1 + 0.45 + 0.05
monthly_jumin_tax = jumin_tax / 12
print("住民税", jumin_tax, "万円, 月額", monthly_jumin_tax, "万円")

annual_zeikin = jumin_tax + hukko + income_tax + nenkin + kokuho
print("あなたは1年で", annual_zeikin, "万円納税しています")

tedori = all_salary - annual_zeikin
print("手取りは", tedori, "万円です")

調整控除額 0
住民税 32.01803833333334 万円, 月額 2.6681698611111115 万円
あなたは1年で 96.115225 万円納税しています
手取りは 455.884775 万円です
